#property strict

input double PriceStepMult = 1.0;
input color  PocColor      = clrOrange;
input int    PocWidth      = 2;

// ─────────────────────────────────────
double step;
double pLow, pHigh;
double volMap[];
int    mapSize = 0;

bool   initialized = false;
string pocName = "DEV_POC_LINE";
int    pocPoint = 0;

// ─────────────────────────────────────
int OnInit()
{
   step = _Point * PriceStepMult;
   ObjectDelete(0, pocName);
   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
void OnDeinit(const int reason)
{
   ObjectDelete(0, pocName);
}

// ─────────────────────────────────────
void OnTick()
{
   double price = SymbolInfoDouble(_Symbol, SYMBOL_BID);
   datetime t   = TimeCurrent();

   //
